In [3]:
!/Users/margaritaberseneva/Downloads/udpipe-1.2.0-bin/bin-osx/udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
/Users/margaritaberseneva/Downloads/udpipe-1.2.0-bin/models/russian-syntagrus-ud-2.4-190531.udpipe \
< test_set.txt > test_set.conllu

Loading UDPipe model: done.


In [148]:
from nltk.parse import DependencyGraph

In [149]:
trees = []
with open('test_set.conllu', encoding='utf-8') as f:
    parsed_sents = f.read().split('\n\n')
    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [150]:
#запишем и посчитаем глаголы
verb_count = {}
for t in trees:
    try:
        g = DependencyGraph(t, top_relation_label='root')
    except:
        continue
    for n in g.nodes:
        if g.nodes[n]['ctag'] == 'VERB':
            if g.nodes[n]['lemma'] in verb_count:
                verb_count[g.nodes[n]['lemma']] += 1
            else:
                verb_count[g.nodes[n]['lemma']] = 1

In [151]:
#оставим только те, которые встречаются больше 50 раз
freq_verbs = []
for verb in verb_count:
    if verb_count[verb] >= 50:
        freq_verbs.append(verb)

In [152]:
#находим Verb+Noun
verb_phrases = []
for t in trees:
    try:
        g = DependencyGraph(t, top_relation_label='root')
        for t in g.triples():
            if t[0][1] == 'VERB' and t[1] == 'obj' and t[2][1] == 'NOUN':
                if t[0][0] in freq_verbs:
                    phrase = [t[0][0], t[2][0]]
                    verb_phrases.append(phrase)
    except:
        continue

In [153]:
verb_phrases[:10]

[['признать', 'договор'],
 ['вынести', 'приговор'],
 ['признать', 'ответственность'],
 ['выплатить', 'компенсации'],
 ['обжаловать', 'санкцию'],
 ['удовлетворить', 'жалобу'],
 ['взыскать', 'деньги'],
 ['обжаловать', 'санкцию'],
 ['оспаривать', 'решение'],
 ['оспаривать', 'решение']]

In [154]:
#оценка метриками
import nltk
from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(verb_phrases)

In [155]:
pmi = finder.nbest(bigram_measures.pmi, 100)
pmi[:10]

[('делать', 'вид'),
 ('отказать', 'жалобы'),
 ('сообщить', 'сведения'),
 ('стать', 'обыски'),
 ('арестовывать', 'счета'),
 ('объявить', 'перерыв'),
 ('назначить', 'наказание'),
 ('назначить', 'сроки'),
 ('обвинить', 'военнослужащих'),
 ('обвинить', 'лидера')]

In [156]:
scores = finder.score_ngrams(bigram_measures.dice)
dice = [x[0] for x in scores]
dice[:10]

[('делать', 'вид'),
 ('объявить', 'перерыв'),
 ('отказать', 'жалобы'),
 ('сообщить', 'сведения'),
 ('стать', 'обыски'),
 ('назначить', 'наказание'),
 ('запретить', 'деятельность'),
 ('арестовывать', 'счета'),
 ('передать', 'дело'),
 ('просить', 'суд')]

In [157]:
logl = finder.nbest(bigram_measures.likelihood_ratio, 100)
logl[:10]

[('обжаловать', 'решение'),
 ('передать', 'дело'),
 ('запретить', 'деятельность'),
 ('назначить', 'наказание'),
 ('рассмотреть', 'вопрос'),
 ('вынести', 'приговор'),
 ('просить', 'суд'),
 ('удовлетворить', 'жалобу'),
 ('выплатить', 'штраф'),
 ('подать', 'иск')]

In [158]:
#смотрим на пересечение трех метрик
intersection = list(set(pmi) & set(dice) & set(logl))
print(len(intersection))

55


In [159]:
intersection[:10]

[('доказать', 'правоту'),
 ('иметь', 'последствия'),
 ('направить', 'документы'),
 ('считать', 'выплату'),
 ('иметь', 'слова'),
 ('предъявить', 'обвинения'),
 ('сообщить', 'сведения'),
 ('передать', 'дело'),
 ('доказать', 'невиновность'),
 ('пройти', 'курс')]

In [160]:
#подгружаем словарь глагольной сочетаемости
with open("verb_coll.txt", "r", encoding='utf-8') as d:
    t = d.read().split('\n')

In [161]:
colls = []
for i in t:
    tri = i.split('\t')
    if tri != ['']:
        colls.append(tri[2])

In [162]:
colls[:10]

['аборт делать',
 'делать аборт',
 'сделать аборт',
 'авария произошла',
 'авария случилась',
 'потерпеть аварию',
 'произошла авария',
 'случилась авария',
 'завоевать авторитет',
 'подорвать авторитет']

In [163]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [164]:
#немного поработаем над коллокациями, оставим только порядок Verb+Noun
new_colls = []
for i in colls:
    words = i.split()
    pos = []
    for w in words:
        p = morph.parse(w)[0]
        tags = str(p.tag).split(',')
        pos.append(tags[0])
    if pos[-1] == 'NOUN':
        pair = []
        for j in i.split():
            pp = morph.parse(j)[0]
            token = pp.normal_form
            pair.append(token)
        s = tuple(pair)
        new_colls.append(s)

In [165]:
new_colls[:10]

[('делать', 'аборт'),
 ('сделать', 'аборт'),
 ('потерпеть', 'авария'),
 ('произойти', 'авария'),
 ('случиться', 'авария'),
 ('завоевать', 'авторитет'),
 ('подорвать', 'авторитет'),
 ('подорвать', 'авторитет'),
 ('пользоваться', 'авторитет'),
 ('пользоваться', 'больший', 'авторитет')]

In [166]:
#пересекаем еще и со словарем глагольной сочетаемости
gold = set(intersection) & set(new_colls)
gold

{('дать', 'заключение'),
 ('делать', 'вид'),
 ('объявить', 'перерыв'),
 ('передать', 'дело'),
 ('предъявить', 'обвинение'),
 ('пройти', 'курс')}

In [168]:
#добавим еще примеры коллокаций
add = set()
s = 'назначить наказание, доказать невиновность, \
            решить судьбу, пройти проверку'
for w in s.split(','):
    add.add(tuple(w.split()))

In [169]:
add

{('доказать', 'невиновность'),
 ('назначить', 'наказание'),
 ('пройти', 'проверку'),
 ('решить', 'судьбу')}

In [170]:
#получаем новое пересечение
new_gold = gold | add

In [171]:
new_gold

{('дать', 'заключение'),
 ('делать', 'вид'),
 ('доказать', 'невиновность'),
 ('назначить', 'наказание'),
 ('объявить', 'перерыв'),
 ('передать', 'дело'),
 ('предъявить', 'обвинение'),
 ('пройти', 'курс'),
 ('пройти', 'проверку'),
 ('решить', 'судьбу')}

In [172]:
#переводим коллокаций из множеств в массивы
collocations = []
c = []
for col in new_gold:
    for w in col:
        c.append(w)
    collocations.append(c)
    c = []

In [173]:
collocations

[['решить', 'судьбу'],
 ['доказать', 'невиновность'],
 ['предъявить', 'обвинение'],
 ['объявить', 'перерыв'],
 ['назначить', 'наказание'],
 ['пройти', 'проверку'],
 ['дать', 'заключение'],
 ['передать', 'дело'],
 ['делать', 'вид'],
 ['пройти', 'курс']]

In [174]:
#считаем коэффициент Спирмена
from scipy import stats

stats.spearmanr(collocations)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


SpearmanrResult(correlation=0.6322217654272672, pvalue=0.049846099509679546)

In [175]:
finder2 = BigramCollocationFinder.from_documents(collocations)
pmi = finder2.nbest(bigram_measures.pmi, 7)
print(stats.spearmanr(pmi))

SpearmanrResult(correlation=0.4642857142857144, pvalue=0.2939341076002517)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [176]:
logl = finder2.nbest(bigram_measures.likelihood_ratio, 7)
print(stats.spearmanr(logl))

SpearmanrResult(correlation=0.4642857142857144, pvalue=0.2939341076002517)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [177]:
scores = finder2.score_ngrams(bigram_measures.dice)
dice = [x[0] for x in scores]
print(stats.spearmanr(dice))

SpearmanrResult(correlation=0.6322217654272672, pvalue=0.049846099509679546)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Метрики разделились на два лагеря, и я, если честно, не совсем знаю, о чем это говорит...